## Final Goal
- Prepare risk-level and location data for visualization of tunnel distribution and risk breakdown.

## Actions
- Convert PDF data to CSV to obtain the number of tunnels by risk level (1–4) and prefecture.
- Prepare data for pie charts by risk level using Datawrapper.
- Acquire latitude and longitude information for each prefecture (prefectural capital) via the GSI API.
- Prepare data to display the number of tunnels owned by each prefecture on a map.

In [3]:
!pip install pdfplumber pandas

In [11]:
import pdfplumber
import pandas as pd


pdf_files = [
    "risklocal.pdf",
    "risknation.pdf",
    "riskprefec.pdf"
]

for pdf in pdf_files:
    rows = []
    with pdfplumber.open(pdf) as pdf_in:
        for page in pdf_in.pages:
            table = page.extract_table()
            if table:
                for row in table[1:]:
                    if not row[0] or "所在する" in row[0] or "判定区分" in row[0]:
                        continue
                    rows.append({
                        "都道府県": row[0],
                        "合計": row[1],
                        "Ⅰ": row[2],
                        "Ⅱ": row[3],
                        "Ⅲ": row[4],
                        "Ⅳ": row[5]
                    })
    if rows:
        df = pd.DataFrame(rows)
        for col in ["合計", "Ⅰ", "Ⅱ", "Ⅲ", "Ⅳ"]:
            df[col] = df[col].str.replace(",", "").astype(int)
        csv_name = pdf.replace(".pdf", ".csv")
        df.to_csv(csv_name, index=False, encoding="utf-8-sig")
        print(f"write{csv_name}")
    else:
        print(f"{pdf}: no table")

risklocal.csv に書き出しました
risknation.csv に書き出しました
riskprefec.csv に書き出しました


In [ ]:
import pandas as pd

In [13]:
riskprefec = pd.read_csv("riskprefec.csv", encoding="utf-8-sig")
risknation = pd.read_csv("risknation.csv", encoding="utf-8-sig")
risklocal = pd.read_csv("risklocal.csv", encoding="utf-8-sig")

merged = (
    riskprefec.set_index("都道府県") +
    risknation.set_index("都道府県") +
    risklocal.set_index("都道府県")
)

merged = merged.reset_index()
merged.to_csv("merged_sum_risk_data.csv", index=False, encoding="utf-8-sig")

print(merged.head())

  都道府県   合計   Ⅰ    Ⅱ    Ⅲ  Ⅳ
0  北海道  929  84  537  306  2
1  青森県  108   0   82   26  0
2  岩手県  611  24  368  219  0
3  宮城県  270  10  133  127  0
4  秋田県  310   2  186  122  0


In [17]:
df = pd.read_csv("merged_sum_risk_data.csv", encoding="utf-8-sig")

df_no_total = df[df["都道府県"] != "合計"]

level_sums = df_no_total[["Ⅰ", "Ⅱ", "Ⅲ", "Ⅳ"]].sum()
total_tunnels = level_sums.sum()

level_ratios = (level_sums / total_tunnels * 100).round(2)

print("合計:", level_ratios.sum())  # 100.0 になる

result = pd.DataFrame({
    "危険度レベル": ["Ⅰ", "Ⅱ", "Ⅲ", "Ⅳ"],
    "件数": level_sums.values,
    "割合(%)": level_ratios.values
})

result.to_csv("risk_level_ratios.csv", index=False, encoding="utf-8-sig")
print(result)

合計: 99.99
  危険度レベル     件数  割合(%)
0      Ⅰ    537   2.63
1      Ⅱ  13696  67.18
2      Ⅲ   6099  29.91
3      Ⅳ     56   0.27


In [20]:
import requests
import urllib.parse
import time

file_path = 'merged_sum_risk_data.csv'
df = pd.read_csv(file_path)

def make_address(pref):
    if pref.endswith('都'):
        return pref + '庁'
    elif pref.endswith('道'):
        return pref + '庁'
    elif pref.endswith('府'):
        return pref + '庁'
    elif pref.endswith('県'):
        return pref + '庁'
    else:
        return pref

def get_lat_lon(address):
    base_url = "https://msearch.gsi.go.jp/address-search/AddressSearch"
    params = {"q": address}
    try:
        response = requests.get(base_url, params=params, timeout=5)
        response.raise_for_status()
        data = response.json()
        # 完全一致するタイトルを探す
        for row in data:
            if row["properties"]["title"] == address:
                coords = row["geometry"]["coordinates"]  # [経度, 緯度]
                return coords[1], coords[0]
        # 完全一致がなければ最初の候補
        if data:
            coords = data[0]["geometry"]["coordinates"]
            return coords[1], coords[0]
    except Exception as e:
        print(f"Error fetching {address}: {e}")
    return None, None

latitudes = []
longitudes = []

for pref in df['都道府県']:
    address = make_address(pref)
    lat, lon = get_lat_lon(address)
    latitudes.append(lat)
    longitudes.append(lon)
    time.sleep(0.2)  # API負荷軽減

df['緯度'] = latitudes
df['経度'] = longitudes

print(df.head())

output_file = 'merged_sum_risk_data_with_latlon_gsi.csv'
df.to_csv(output_file, index=False, encoding='utf-8-sig')


  都道府県   合計   Ⅰ    Ⅱ    Ⅲ  Ⅳ         緯度          経度
0  北海道  929  84  537  306  2  43.064323  141.346878
1  青森県  108   0   82   26  0  40.824589  140.740555
2  岩手県  611  24  368  219  0  39.703526  141.152695
3  宮城県  270  10  133  127  0  38.268578  140.872070
4  秋田県  310   2  186  122  0  39.718624  140.102386
